In [1]:
from utils import *
from vgg16 import Vgg16

Using Theano backend.


In [2]:
path = "../data/state_farm/sample"

In [3]:
vgg = Vgg16()
vgg.compile()

/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [4]:
batches = vgg.get_batches(path + '/train', batch_size=16)
val_batches = vgg.get_batches(path + '/valid', batch_size = 16)

Found 2000 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [5]:
layers = vgg.model.layers
last_conv_index = [index for index, layer in enumerate(layers) if type(layer) is Convolution2D][-1]

In [6]:
conv_layers = layers[:last_conv_index+1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_index+1:]

In [7]:
trn_classes = batches.classes
val_classes = val_batches.classes
trn_labels = onehot(trn_classes)
val_labels = onehot(val_classes)

In [8]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [9]:
trn_features= conv_model.predict_generator(batches, batches.nb_sample)

In [13]:
val_features.shape
trn_features.shape
trn_classes.shape
trn_labels.shape

(2000, 10)

In [15]:
save_array(path + '/models/conv_val_features.bc', val_features)
save_array(path + '/models/conv_trn_features.bc', trn_features)
save_array(path + '/models/trn_classes.bc', trn_classes)
save_array(path + '/models/val_classes.bc', val_classes)
save_array(path + '/models/trn_labels.bc', trn_labels)
save_array(path + '/models/val_labels.bc', val_labels)

In [16]:
opt = RMSprop(lr=0.00001, rho=0.7)
# opt = RMSprop(lr=0.0005)

In [17]:
def get_fc_model():
    last_output_shape = conv_layers[-1].output_shape[1:]
    # Can we remove the two dropout layers?
    model = Sequential([
        MaxPooling2D(input_shape=last_output_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.2),
        Dense(4096, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ])
#     for l1, l2 in zip(model.layers, fc_layers):
#         l1.set_weights(proc_weights(l2))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
fc_model = get_fc_model()
fc_model

In [23]:
fc_model.optimizer.lr = 0.0001

In [24]:
fc_model.fit(trn_features, trn_labels, nb_epoch=5, batch_size=32, validation_data=(val_features, val_labels))

Train on 2000 samples, validate on 200 samples
Epoch 1/5
2000/2000 [==============================] - 75s - loss: 0.8375 - acc: 0.8445 - val_loss: 10.0188 - val_acc: 0.0750
Epoch 2/5
2000/2000 [==============================] - 75s - loss: 0.8264 - acc: 0.8570 - val_loss: 10.3470 - val_acc: 0.0900
Epoch 3/5
2000/2000 [==============================] - 75s - loss: 0.7108 - acc: 0.8640 - val_loss: 10.3536 - val_acc: 0.0950
Epoch 4/5
2000/2000 [==============================] - 75s - loss: 0.6032 - acc: 0.8740 - val_loss: 10.2092 - val_acc: 0.0800
Epoch 5/5
2000/2000 [==============================] - 78s - loss: 0.5211 - acc: 0.8890 - val_loss: 10.5082 - val_acc: 0.0800
